## Mask RCNN Training
Training Mask RCNN (standard or lightweight) with custom dataset.

In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model_final as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

## Configurations

In [2]:
class Config(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024
    BACKBONE = "resnet18"
    NUM_CLASSES = 1 + 5
    # LEARNING_RATE = 0.0001
    # TRAIN_BN = False
    # USE_MINI_MASK = False
    # DETECTION_MIN_CONFIDENCE = 0.7
    # COMPUTE_BACKBONE_SHAPE = False 
    STEPS_PER_EPOCH = 300
    VALIDATION_STEPS = 30
    WEIGHT_DECAY = 0.001

config = Config()
config.display()


Configurations:
BACKBONE                       resnet18
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         True
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                18
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

## Load Model

In [3]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
# COCO_MODEL_PATH = model.find_last()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [4]:
model.load_weights(r"model\dir", by_name=True)

In [4]:
model.keras_model.summary()

Model: "mask_rcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, None, None,  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNorm)            (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________

## Loading Custom Data

In [5]:
import coco_RBP
config = coco_RBP.CocoConfig()
COCO_DIR = r"data\dir"  # TODO: enter value here

In [6]:
train_dataset = coco_RBP.CocoDataset()
train_dataset.load_coco(COCO_DIR, "val_coco")

train_dataset.prepare()

print("Image Count: {}".format(len(train_dataset.image_ids)))
print("Class Count: {}".format(train_dataset.num_classes))
for i, info in enumerate(train_dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Image Count: 60
Class Count: 6
  0. BG                                                
  1. pilers                                            
  2. screwdriver                                       
  3. wheel                                             
  4. holder                                            
  5. cylinder                                          


In [7]:
val_dataset = coco_RBP.CocoDataset()
val_dataset.load_coco(COCO_DIR, "test_coco")

val_dataset.prepare()

print("Image Count: {}".format(len(val_dataset.image_ids)))
print("Class Count: {}".format(val_dataset.num_classes))
for i, info in enumerate(train_dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Image Count: 60
Class Count: 6
  0. BG                                                
  1. pilers                                            
  2. screwdriver                                       
  3. wheel                                             
  4. holder                                            
  5. cylinder                                          


In [8]:
import time

## Training

In [ ]:
start_train = time.time()
model.train(train_dataset,val_dataset,learning_rate=Config.LEARNING_RATE, layers='all',
            epochs=5) #4 -KYLE- I edited this to help troubleshoot the kernel : change it back for real training )

end_train = time.time()
minutes = round((end_train - start_train) / 60, 2)
print(f'Training took {minutes} minutes')


## Save Weightings

In [ ]:
model.keras_model.save_weights('./re-train_0429.h5')